In [30]:
#Dependencies and Setup
import pandas as pd
import math
import numpy as np

#File to Load
train_data= "clean_data_rs/ptv_metro_train_station.csv"
listing_data= "clean_data_rs/airbnbFINAL_Swobabika.csv"

#files and store into Pandas data frame
station_df = pd.read_csv(train_data) 
station_df.head()

,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,TICKETZONE,ROUTEUSSP
0,19967,Anstey Railway Station (Brunswick),-37.761242,144.960684,1,Upfield
1,19968,Brunswick Railway Station (Brunswick),-37.767721,144.959587,1,Upfield
2,19969,Jewell Railway Station (Brunswick),-37.774987,144.958717,1,Upfield
3,19970,Royal Park Railway Station (Parkville),-37.781193,144.952301,1,Upfield
4,19971,Flemington Bridge Railway Station (North Melbo...,-37.788140,144.939323,1,Upfield


In [31]:
listing_df = pd.read_csv(listing_data, low_memory=False)
listing_df.head()

,id,host_id,host_name,host_is_superhost,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,...,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd
0,12936,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,...,15,94,9,10,94,9,9,9,0.15,5.205174
1,47100,212071,Loren,t,4,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,...,176,98,10,10,98,10,10,10,1.75,3.649571
2,50492,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,...,44,97,10,10,97,10,9,10,0.60,5.437003
3,51592,190879,Michelle,t,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,...,196,99,10,9,99,10,10,10,2.07,0.072335
4,66754,50121,Frank & Vince,t,17,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,...,43,95,9,10,95,10,10,9,0.46,2.794548


In [32]:
station_df.dtypes

STOP_ID         int64
STOP_NAME      object
LATITUDE      float64
LONGITUDE     float64
TICKETZONE     object
ROUTEUSSP      object
dtype: object

In [33]:
listing_df.dtypes

id                               int64
host_id                          int64
host_name                       object
host_is_superhost               object
host_total_listings_count        int64
street                          object
neighbourhood_cleansed          object
city                            object
state                           object
zipcode                         object
latitude                       float64
longitude                      float64
property_type                   object
accommodates                     int64
bathrooms                      float64
bedrooms                       float64
beds                           float64
bed_type                        object
amenities                       object
price                          float64
weekly_price                   float64
monthly_price                  float64
security_deposit               float64
cleaning_fee                   float64
guests_included                  int64
extra_people             

In [34]:
from math import radians, cos, sin, asin, sqrt
def dist(lat1, long1, lat2, long2):

    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [35]:
def find_nearest(lat, long):
    distances = station_df.apply(
        lambda row: dist(lat, long, row['LATITUDE'], row['LONGITUDE']), 
        axis=1)
    return station_df.loc[distances.idxmin(), 'STOP_NAME']

# append the station name back to the listings file
listing_df['STOP_NAME'] = listing_df.apply(lambda row: find_nearest(row['latitude'], row['longitude']), axis=1)
# To check the data frame if it has a new column of station name (for each and every listing)
listing_df.head()

,id,host_id,host_name,host_is_superhost,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd,STOP_NAME
0,12936,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,...,94,9,10,94,9,9,9,0.15,5.205174,Windsor Railway Station (Windsor)
1,47100,212071,Loren,t,4,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,...,98,10,10,98,10,10,10,1.75,3.649571,Burnley Railway Station (Burnley)
2,50492,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,...,97,10,10,97,10,9,10,0.60,5.437003,Windsor Railway Station (Windsor)
3,51592,190879,Michelle,t,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,...,99,10,9,99,10,10,10,2.07,0.072335,Flinders Street Railway Station (Melbourne City)
4,66754,50121,Frank & Vince,t,17,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,...,95,9,10,95,10,10,9,0.46,2.794548,Richmond Railway Station (Richmond)


In [36]:
listing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9308 entries, 0 to 9307
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           9308 non-null   int64  
 1   host_id                      9308 non-null   int64  
 2   host_name                    9308 non-null   object 
 3   host_is_superhost            9308 non-null   object 
 4   host_total_listings_count    9308 non-null   int64  
 5   street                       9308 non-null   object 
 6   neighbourhood_cleansed       9308 non-null   object 
 7   city                         9308 non-null   object 
 8   state                        9308 non-null   object 
 9   zipcode                      9308 non-null   object 
 10  latitude                     9308 non-null   float64
 11  longitude                    9308 non-null   float64
 12  property_type                9308 non-null   object 
 13  accommodates      

In [37]:
# create one dataframe with the new info
listing_final_df = pd.merge(listing_df,station_df[['STOP_NAME','LATITUDE','LONGITUDE']],on='STOP_NAME', how='left')
 
listing_final_df.head()

,id,host_id,host_name,host_is_superhost,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd,STOP_NAME,LATITUDE,LONGITUDE
0,12936,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,...,10,94,9,9,9,0.15,5.205174,Windsor Railway Station (Windsor),-37.856053,144.992035
1,47100,212071,Loren,t,4,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,...,10,98,10,10,10,1.75,3.649571,Burnley Railway Station (Burnley),-37.827560,145.007555
2,50492,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,...,10,97,10,9,10,0.60,5.437003,Windsor Railway Station (Windsor),-37.856053,144.992035
3,51592,190879,Michelle,t,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,...,9,99,10,10,10,2.07,0.072335,Flinders Street Railway Station (Melbourne City),-37.818305,144.966964
4,66754,50121,Frank & Vince,t,17,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,...,10,95,10,10,9,0.46,2.794548,Richmond Railway Station (Richmond),-37.824074,144.990164


In [38]:
#    Calculate the great circle distance between two points 
#    on the earth (specified in decimal degrees)

from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [39]:
# Creating a new column to generate the output by passing lat long information to Haversine Equation
listing_final_df['station_distance'] = [haversine(listing_final_df.longitude[i],listing_final_df.latitude[i],listing_final_df.LONGITUDE[i],listing_final_df.LATITUDE[i]) for i in range(len(listing_final_df))]
listing_final_df['station_distance'] = listing_final_df['station_distance'].round(decimals=1)
# Printing the data table 
listing_final_df.head()


,id,host_id,host_name,host_is_superhost,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd,STOP_NAME,LATITUDE,LONGITUDE,station_distance
0,12936,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,...,94,9,9,9,0.15,5.205174,Windsor Railway Station (Windsor),-37.856053,144.992035,1.4
1,47100,212071,Loren,t,4,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,...,98,10,10,10,1.75,3.649571,Burnley Railway Station (Burnley),-37.827560,145.007555,1.1
2,50492,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,...,97,10,9,10,0.60,5.437003,Windsor Railway Station (Windsor),-37.856053,144.992035,1.2
3,51592,190879,Michelle,t,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,...,99,10,10,10,2.07,0.072335,Flinders Street Railway Station (Melbourne City),-37.818305,144.966964,0.5
4,66754,50121,Frank & Vince,t,17,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,...,95,10,10,9,0.46,2.794548,Richmond Railway Station (Richmond),-37.824074,144.990164,0.5


In [40]:
listing_final_df[['id', 'latitude', 'longitude', 'STOP_NAME', 'LATITUDE', 'LONGITUDE', 'station_distance']].to_csv('listing_station_coords.csv')